# Array
The array module handles small arrays that hold numerical data that needs to be used in a computation of a kernel function. Unlike [gridfunctions](gridfunction.ipynb) that use a `u[i,j]` syntax for accessing the two or more indices. The array syntax depends on the target language. For example, if the generated code is `C`, then the array uses C-style array formatting. In this case, it is easy to distinguish gridfunctions from arrays.

First, we import the array module.

In [1]:
from openfd.alpha import array

Next, before we can construct an array, we need some data to store. For this purpose, we use a numpy array

In [2]:
import numpy
data = numpy.array([[1.1, 0.0], [0.0, 1.2]])
data

array([[1.1, 0. ],
       [0. , 1.2]])

Now, we can construct the symbolic array and fill it using the data above. Throughout this notebook, we use a `C`-array.

In [3]:
a = array.CArray('a', data)

## Symbolic access
A symbolic access outputs an element of the array in a symbolic format the is determined by the target language (in this case `C`)

In [4]:
a[1,1]

a[1][1]

In the above, note how the array access has changed from `[1,1]` to `[1][1]`.

In contrast to [gridfunctions](gridfunction.ipynb), the number of indices must match the dimensionality of the array. The following example raises an exception.

In [5]:
#a[2,2,2] # Uncomment to raise an exception

## Value access
A value access returns the array value itself and is accomplished by

In [6]:
a(1,1)

1.2

## Initialization code
In a kernel function, the array needs to be initialized before its use. The code for array initialization is obtained by using the appropriate Sympy code printer to print the array. For example, to initialize a C-array, we import the `ccode` printer.

In [7]:
from sympy import ccode
ccode(a)

'const float a[2][2] = {{1.1, 0}, {0, 1.2}};'

## Extensions
By default, the array initialization assumes floating-point output. To change this format, we can override the method `dec_str` as follows.

In [8]:
class CArrayInt(array.CArray):
    def dec_str(self):
        self.rettype = 'const int' # return type
        self.format = '%d' # formatting string for each value
    def __call__(self, *index):
        return int(self.data[index])

The above example will output any of the data values as integers.  

In [9]:
b = CArrayInt('b', data)
ccode(b)

'const int b[2][2] = {{1, 0}, {0, 1}};'

In [10]:
b(0,0)

1

Let's add Arrays together!

In [11]:
a = array.CArray('a', data)
b = array.CArray('b', data)
c = a + b

This works fine!

In [12]:
c

a + b

However, this is wrong!

In [13]:
ccode(c)

'const float a[2][2] = {{1.1, 0}, {0, 1.2}};\n+ const float b[2][2] = {{1.1, 0}, {0, 1.2}};'

And we can't accces the data

In [14]:
c(0,0)

TypeError: 'Add' object is not callable

If the user tries to perform operations on Arrays, it should throw an exception. 

How Arrays and Gridfunction can interact in an Expression ?

My line of thinking went like this: If you want to get the initialization code for the array, you would call it for one array at a time. For example:

In [ ]:
ccode(a)

In [ ]:
ccode(b)

However, it would be a lot cleaner if you only had to write the expression and it could pull the initialization code from it. I think the best solution is to use another function for initializing the arrays. That is `ccode(a)` should only print `a`. Perhaps `init = Initialize(a+b)` and then `ccode(init`) would produce the correct initialization code:
```
>>> init = Initialize(a+b)
>>> ccode(init)
const float a[2][2] = {{1.1, 0}, {0, 1.2}};
const float b[2][2] = {{1.1, 0}, {0, 1.2}};
```
We could assign an attribute `is_initializable` and then implement a function that handles initialization. The Initialization class would be responsible for building a list of all objects that require initialization.


To make grid functions and arrays work together, they need to be wrap using `Expression`.

In [ ]:
from openfd.alpha import Expression
expr = Expression(a + b)
expr[0,0]

However, now we can't access their data as you said. Maybe we should add `__call__` to expression as well? A call on a grid function could invoke the same behavior as `__getitem__`. That would solve the problem.

In [ ]:
expr(0,0)